# Random Forest Regressor 

## Mục tiêu:
- Xây dựng mô hình Random Forest Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Lưu mô hình đã huấn luyện


## Giới thiệu

Là tập hợp nhiều cây quyết định độc lập (bagging ensemble).

## Ưu điểm

- Giảm overfitting so với Decision Tree.

- Ổn định, mạnh mẽ.

- Tự động đánh giá tầm quan trọng của biến.

- Xử lý tốt dữ liệu lớn.

## Nhược điểm

- Không dễ diễn giải như 1 cây duy nhất.

- Cần nhiều tài nguyên tính toán.

## Bước 1 - Import các thư viện cần thiết

In [ ]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import os

# Thư viện sklearn cho mô hình và đánh giá
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [ ]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [ ]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

## Bước 4 - Xây dựng mô hình Random Forest 

## Bước 5 - Tối ưu hóa siêu tham số với GridSearchCV

Sử dụng GridSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình Random Forest.

### Các siêu tham số cần tối ưu:
- **n_estimators:** Số lượng cây trong rừng
- **max_depth:** Độ sâu tối đa của mỗi cây
- **min_samples_split:** Số mẫu tối thiểu để chia một nút
- **min_samples_leaf:** Số mẫu tối thiểu ở một nút lá
- **max_features:** Số lượng đặc trưng tối đa xem xét khi chia
- **bootstrap:** Có sử dụng bootstrap sampling hay không

In [ ]:
# Định nghĩa không gian siêu tham số để tìm kiếm
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [10, 15, 20, 25, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

print("="*60)
print("TÌM KIẾM SIÊU THAM SỐ TỐI ƯU")
print("="*60)
print(f"Số lượng tổ hợp tham số: {np.prod([len(v) for v in param_grid.values()])}")
print("Phương pháp: GridSearchCV với 5-Fold Cross-Validation")
print("Metric đánh giá: Negative RMSE (neg_root_mean_squared_error)")
print("\nĐang thực hiện tìm kiếm...")
print("⚠️  Lưu ý: Quá trình này có thể mất nhiều thời gian (10-30 phút)")
print("="*60)

# Tạo KFold với 5 folds
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Tạo GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    cv=kfold,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

# Thực hiện tìm kiếm
grid_search.fit(X_train, y_train)

print("\n✓ Hoàn thành tìm kiếm!")

## Bước 6 - Phân tích kết quả GridSearchCV

Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [ ]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_rf = grid_search.best_estimator_
best_params = grid_search.best_params_
best_cv_score = -grid_search.best_score_  # Chuyển về dương (RMSE)

print("="*60)
print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

print(f"\nRMSE Cross-Validation (5-fold): {best_cv_score:.4f} năm")
print("="*60)

# Thông tin về mô hình tối ưu
print(f"\nSố lượng cây: {best_rf.n_estimators}")
print(f"Số đặc trưng xem xét mỗi lần chia: {best_rf.max_features}")
print(f"Bootstrap sampling: {best_rf.bootstrap}")

## Bước 7 - Đánh giá mô hình tối ưu trên tập Validation

Sử dụng mô hình đã tối ưu để dự đoán trên tập validation và tính toán các metrics.

In [ ]:
# Dự đoán trên tập train và validation
print("Đang thực hiện dự đoán...")
y_train_pred = best_rf.predict(X_train)
y_val_pred = best_rf.predict(X_val)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

# Tính toán các metrics cho tập validation
val_mae = mean_absolute_error(y_val, y_val_pred)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
val_r2 = r2_score(y_val, y_val_pred)

print("="*60)
print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print(f"\nTập TRAIN:")
print(f"  MAE:  {train_mae:.4f} năm")
print(f"  RMSE: {train_rmse:.4f} năm")
print(f"  R²:   {train_r2:.4f}")

print(f"\nTập VALIDATION:")
print(f"  MAE:  {val_mae:.4f} năm")
print(f"  RMSE: {val_rmse:.4f} năm")
print(f"  R²:   {val_r2:.4f}")

print("\n" + "="*60)
print("SO SÁNH VỚI BASELINE")
print("="*60)
print(f"Cải thiện RMSE trên validation: {val_rmse_baseline - val_rmse:.4f} năm")
print(f"Cải thiện R² trên validation:   {val_r2 - val_r2_baseline:.4f}")
print("="*60)

## Bước 8 - Phân tích Feature Importance

Phân tích mức độ quan trọng của từng đặc trưng trong việc dự đoán tuổi thọ trung bình. Random Forest cung cấp feature importance đáng tin cậy hơn Decision Tree đơn lẻ.

In [ ]:
# Lấy feature importance
feature_importance = best_rf.feature_importances_

# Tạo DataFrame để sắp xếp
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("="*60)
print("ĐỘ QUAN TRỌNG CỦA CÁC ĐẶC TRƯNG (Feature Importance)")
print("="*60)
for idx, row in importance_df.iterrows():
    print(f"{row['feature']:20s}: {row['importance']:.4f} {'█' * int(row['importance'] * 100)}")
print("="*60)

# Vẽ biểu đồ feature importance
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance'], color='forestgreen', edgecolor='black')
plt.xlabel('Độ quan trọng', fontsize=12, fontweight='bold')
plt.ylabel('Đặc trưng', fontsize=12, fontweight='bold')
plt.title('Feature Importance - Random Forest Regressor', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()

# Tạo thư mục nếu chưa có
os.makedirs('../visualization/random_forest', exist_ok=True)
plt.savefig('../visualization/random_forest/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ feature importance tại: ../visualization/random_forest/feature_importance.png")

## Bước 9 - Trực quan hóa kết quả dự đoán

Vẽ biểu đồ so sánh giá trị thực tế và giá trị dự đoán trên tập validation.

In [ ]:
# Tạo figure với 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Subplot 1: Scatter plot - Predicted vs Actual (Validation)
axes[0].scatter(y_val, y_val_pred, alpha=0.5, color='forestgreen', edgecolor='black', linewidth=0.5)
axes[0].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2, label='Dự đoán hoàn hảo')
axes[0].set_xlabel('Tuổi thọ thực tế (năm)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Tuổi thọ dự đoán (năm)', fontsize=12, fontweight='bold')
axes[0].set_title(f'Dự đoán vs Thực tế (Validation)\nR² = {val_r2:.4f}, RMSE = {val_rmse:.4f}', 
                  fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Subplot 2: Residual plot
residuals = y_val - y_val_pred
axes[1].scatter(y_val_pred, residuals, alpha=0.5, color='darkorange', edgecolor='black', linewidth=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Tuổi thọ dự đoán (năm)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Residuals (Thực tế - Dự đoán)', fontsize=12, fontweight='bold')
axes[1].set_title('Biểu đồ Residuals (Validation)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../visualization/random_forest/predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Đã lưu biểu đồ dự đoán tại: ../visualization/random_forest/predictions.png")

## Bước 10 - Phân tích Cross-Validation scores

Trực quan hóa phân bố điểm số RMSE từ 5-Fold Cross-Validation để đánh giá độ ổn định của mô hình.

In [ ]:
# Lấy kết quả cross-validation từ GridSearchCV
cv_results = grid_search.cv_results_
best_index = grid_search.best_index_

# Lấy scores của mô hình tốt nhất qua các folds
best_cv_scores = []
for i in range(5):  # 5 folds
    fold_score = -cv_results[f'split{i}_test_score'][best_index]  # Chuyển về RMSE dương
    best_cv_scores.append(fold_score)

print("="*60)
print("KẾT QUẢ 5-FOLD CROSS-VALIDATION")
print("="*60)
for i, score in enumerate(best_cv_scores, 1):
    print(f"Fold {i}: RMSE = {score:.4f} năm")

print(f"\nTrung bình: {np.mean(best_cv_scores):.4f} năm")
print(f"Độ lệch chuẩn: {np.std(best_cv_scores):.4f} năm")
print(f"Min: {np.min(best_cv_scores):.4f} năm")
print(f"Max: {np.max(best_cv_scores):.4f} năm")
print("="*60)

# Vẽ biểu đồ
plt.figure(figsize=(10, 6))
plt.plot(range(1, 6), best_cv_scores, marker='o', linewidth=2, markersize=10, color='forestgreen')
plt.axhline(y=np.mean(best_cv_scores), color='red', linestyle='--', linewidth=2, 
            label=f'Trung bình = {np.mean(best_cv_scores):.4f}')
plt.fill_between(range(1, 6), 
                 np.mean(best_cv_scores) - np.std(best_cv_scores),
                 np.mean(best_cv_scores) + np.std(best_cv_scores),
                 alpha=0.2, color='red', label=f'± 1 std = {np.std(best_cv_scores):.4f}')
plt.xlabel('Fold', fontsize=12, fontweight='bold')
plt.ylabel('RMSE (năm)', fontsize=12, fontweight='bold')
plt.title('RMSE qua các Folds trong Cross-Validation', fontsize=14, fontweight='bold')
plt.xticks(range(1, 6))
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../visualization/random_forest/cv_scores.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ CV scores tại: ../visualization/random_forest/cv_scores.png")

# Đánh giá độ ổn định
if np.std(best_cv_scores) < 0.5:
    print("\n✓ Mô hình rất ổn định (độ lệch chuẩn < 0.5)")
elif np.std(best_cv_scores) < 1.0:
    print("\n✓ Mô hình khá ổn định (độ lệch chuẩn < 1.0)")
else:
    print("\n⚠️  Mô hình có độ biến động cao (độ lệch chuẩn ≥ 1.0)")

## Bước 11 - Phân tích tầm quan trọng của từng cây

Random Forest bao gồm nhiều cây quyết định. Phân tích sự đóng góp của từng cây vào dự đoán cuối cùng.

In [ ]:
# Lấy dự đoán từ từng cây riêng lẻ
individual_predictions = []
for tree in best_rf.estimators_[:min(10, len(best_rf.estimators_))]:  # Lấy 10 cây đầu tiên
    individual_predictions.append(tree.predict(X_val[:100]))  # Dự đoán 100 mẫu đầu

individual_predictions = np.array(individual_predictions)

# Vẽ biểu đồ
plt.figure(figsize=(14, 6))

# Subplot 1: Dự đoán của các cây riêng lẻ
for i in range(len(individual_predictions)):
    plt.plot(range(100), individual_predictions[i], alpha=0.3, linewidth=1, color='gray')

# Dự đoán trung bình (của Random Forest)
plt.plot(range(100), y_val_pred[:100], linewidth=2, color='forestgreen', label='Random Forest (trung bình)', zorder=10)

# Giá trị thực tế
plt.plot(range(100), y_val.values[:100], linewidth=2, color='red', label='Thực tế', linestyle='--', zorder=11)

plt.xlabel('Chỉ số mẫu', fontsize=12, fontweight='bold')
plt.ylabel('Tuổi thọ (năm)', fontsize=12, fontweight='bold')
plt.title('So sánh Dự đoán của các Cây riêng lẻ vs Random Forest', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../visualization/random_forest/individual_trees.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Đã lưu biểu đồ individual trees tại: ../visualization/random_forest/individual_trees.png")
print("\n📊 Nhận xét: Các đường màu xám thể hiện dự đoán của từng cây riêng lẻ.")
print("            Đường màu xanh lá là trung bình (Random Forest) - ổn định và chính xác hơn.")

## Bước 12 - Đánh giá mô hình trên tập Test

Đánh giá hiệu suất cuối cùng của mô hình trên tập test (dữ liệu chưa từng thấy).

In [ ]:
# Dự đoán trên tập test
print("Đang dự đoán trên tập test...")
y_test_pred = best_rf.predict(X_test)

# Tính toán các metrics cho tập test
test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)

print("="*60)
print("KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST")
print("="*60)
print(f"\nMAE:  {test_mae:.4f} năm")
print(f"RMSE: {test_rmse:.4f} năm")
print(f"R²:   {test_r2:.4f}")
print("="*60)

# Tạo bảng so sánh
comparison_df = pd.DataFrame({
    'Tập dữ liệu': ['Train', 'Validation', 'Test'],
    'MAE': [train_mae, val_mae, test_mae],
    'RMSE': [train_rmse, val_rmse, test_rmse],
    'R²': [train_r2, val_r2, test_r2]
})

print("\nBẢNG SO SÁNH KẾT QUẢ:")
print(comparison_df.to_string(index=False))
print("="*60)

# Vẽ biểu đồ so sánh
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['MAE', 'RMSE', 'R²']
datasets = ['Train', 'Validation', 'Test']
colors = ['#3498db', '#e74c3c', '#2ecc71']

for idx, metric in enumerate(metrics):
    values = comparison_df[metric].values
    axes[idx].bar(datasets, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
    axes[idx].set_ylabel(metric, fontsize=12, fontweight='bold')
    axes[idx].set_title(f'{metric} trên các tập dữ liệu', fontsize=13, fontweight='bold')
    axes[idx].grid(True, alpha=0.3, axis='y')
    
    # Thêm giá trị lên đầu cột
    for i, v in enumerate(values):
        axes[idx].text(i, v, f'{v:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('../visualization/random_forest/comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ so sánh tại: ../visualization/random_forest/comparison.png")

## Bước 13 - Lưu mô hình đã huấn luyện

Lưu mô hình Random Forest đã tối ưu vào file để sử dụng sau này.

In [ ]:
# Tạo thư mục saved_models nếu chưa có
os.makedirs('../saved_models', exist_ok=True)

# Lưu mô hình
model_path = '../saved_models/random_forest.pkl'
joblib.dump(best_rf, model_path)

print("="*60)
print("LƯU MÔ HÌNH")
print("="*60)
print(f"✓ Đã lưu mô hình tại: {model_path}")
print(f"Kích thước file: {os.path.getsize(model_path) / 1024:.2f} KB")
print("="*60)

# Kiểm tra tải lại mô hình
loaded_model = joblib.load(model_path)
y_test_pred_loaded = loaded_model.predict(X_test)
test_rmse_loaded = np.sqrt(mean_squared_error(y_test, y_test_pred_loaded))

print(f"\n✓ Kiểm tra tải lại mô hình:")
print(f"  RMSE trên test (mô hình gốc):      {test_rmse:.4f} năm")
print(f"  RMSE trên test (mô hình đã tải):   {test_rmse_loaded:.4f} năm")
print(f"  Chênh lệch: {abs(test_rmse - test_rmse_loaded):.10f} (rất nhỏ → OK)")

## Bước 14 - Phân tích sâu về Overfitting/Underfitting

Phân tích learning curve để hiểu rõ hơn về hành vi của mô hình.

In [ ]:
from sklearn.model_selection import learning_curve

# Tính toán learning curve
print("Đang tính toán learning curve (có thể mất vài phút)...")
train_sizes, train_scores, val_scores = learning_curve(
    best_rf, X_train, y_train,
    cv=5,
    scoring='neg_root_mean_squared_error',
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1,
    random_state=42
)

# Chuyển sang RMSE dương và tính mean, std
train_scores_mean = -train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
val_scores_mean = -val_scores.mean(axis=1)
val_scores_std = val_scores.std(axis=1)

# Vẽ learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, 'o-', color='blue', linewidth=2, markersize=8, label='RMSE Train')
plt.plot(train_sizes, val_scores_mean, 'o-', color='red', linewidth=2, markersize=8, label='RMSE Validation')

plt.fill_between(train_sizes, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std,
                 alpha=0.2, color='blue')
plt.fill_between(train_sizes,
                 val_scores_mean - val_scores_std,
                 val_scores_mean + val_scores_std,
                 alpha=0.2, color='red')

plt.xlabel('Số lượng mẫu huấn luyện', fontsize=12, fontweight='bold')
plt.ylabel('RMSE (năm)', fontsize=12, fontweight='bold')
plt.title('Learning Curve - Random Forest Regressor', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../visualization/random_forest/learning_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Đã lưu learning curve tại: ../visualization/random_forest/learning_curve.png")

# Phân tích
print("\n" + "="*60)
print("PHÂN TÍCH LEARNING CURVE")
print("="*60)
gap = train_scores_mean[-1] - val_scores_mean[-1]
if gap < 0.3:
    print("✓ Khoảng cách giữa train và validation RMSE rất nhỏ")
    print("  → Mô hình cân bằng rất tốt, Random Forest giảm overfitting hiệu quả")
elif gap < 0.5:
    print("✓ Khoảng cách giữa train và validation RMSE nhỏ")
    print("  → Mô hình cân bằng tốt")
elif gap < 1.0:
    print("⚠ Khoảng cách giữa train và validation RMSE trung bình")
    print("  → Có dấu hiệu overfitting nhẹ")
else:
    print("✗ Khoảng cách giữa train và validation RMSE lớn")
    print("  → Mô hình bị overfitting")
print("="*60)

## Bước 15 - Phân tích Out-of-Bag (OOB) Score

Out-of-Bag (OOB) error là một phương pháp validation đặc biệt của Random Forest, sử dụng các mẫu không được chọn trong bootstrap sampling.

In [ ]:
# Huấn luyện lại mô hình với oob_score=True nếu bootstrap=True
if best_rf.bootstrap:
    print("Đang tính toán OOB score...")
    rf_oob = RandomForestRegressor(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        max_features=best_params['max_features'],
        bootstrap=True,
        oob_score=True,
        random_state=42,
        n_jobs=-1
    )
    
    rf_oob.fit(X_train, y_train)
    oob_score = rf_oob.oob_score_
    
    print("="*60)
    print("OUT-OF-BAG (OOB) SCORE")
    print("="*60)
    print(f"OOB R² Score: {oob_score:.4f}")
    print("\nOOB score cung cấp ước lượng hiệu suất mô hình mà không cần")
    print("tập validation riêng biệt, sử dụng các mẫu không được chọn")
    print("trong quá trình bootstrap sampling của mỗi cây.")
    print("="*60)
    
    # So sánh OOB với validation score
    print(f"\nSo sánh:")
    print(f"  OOB R²:        {oob_score:.4f}")
    print(f"  Validation R²: {val_r2:.4f}")
    print(f"  Chênh lệch:    {abs(oob_score - val_r2):.4f}")
else:
    print("="*60)
    print("Mô hình không sử dụng bootstrap, OOB score không khả dụng.")
    print("="*60)

## Bước 16 - Tạo báo cáo tổng kết

Tổng hợp tất cả thông tin quan trọng về mô hình vào một báo cáo.

In [ ]:
# Tạo báo cáo
report = f"""
{'='*70}
              BÁO CÁO MÔ HÌNH RANDOM FOREST REGRESSOR
{'='*70}

1. THÔNG TIN MÔ HÌNH:
   - Loại mô hình: Random Forest Regressor
   - Thư viện: scikit-learn
   - Thời gian tạo: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

2. SIÊU THAM SỐ TỐI ƯU:
   - n_estimators: {best_params['n_estimators']}
   - max_depth: {best_params['max_depth']}
   - min_samples_split: {best_params['min_samples_split']}
   - min_samples_leaf: {best_params['min_samples_leaf']}
   - max_features: {best_params['max_features']}
   - bootstrap: {best_params['bootstrap']}

3. ĐẶC ĐIỂM MÔ HÌNH:
   - Số lượng cây: {best_rf.n_estimators}
   - Số đặc trưng: {len(feature_cols)}

4. KẾT QUẢ ĐÁNH GIÁ:
   
   Tập TRAIN:
   - MAE:  {train_mae:.4f} năm
   - RMSE: {train_rmse:.4f} năm
   - R²:   {train_r2:.4f}
   
   Tập VALIDATION:
   - MAE:  {val_mae:.4f} năm
   - RMSE: {val_rmse:.4f} năm
   - R²:   {val_r2:.4f}
   
   Tập TEST (Final):
   - MAE:  {test_mae:.4f} năm
   - RMSE: {test_rmse:.4f} năm
   - R²:   {test_r2:.4f}

5. CROSS-VALIDATION (5-Fold):
   - RMSE trung bình: {np.mean(best_cv_scores):.4f} năm
   - Độ lệch chuẩn: {np.std(best_cv_scores):.4f} năm
   - RMSE tối thiểu: {np.min(best_cv_scores):.4f} năm
   - RMSE tối đa: {np.max(best_cv_scores):.4f} năm

6. SO SÁNH VỚI BASELINE:
   - Cải thiện RMSE validation: {val_rmse_baseline - val_rmse:.4f} năm
   - Cải thiện R² validation: {val_r2 - val_r2_baseline:.4f}

7. TOP 5 ĐẶC TRƯNG QUAN TRỌNG NHẤT:
"""

for i, (idx, row) in enumerate(importance_df.head(5).iterrows(), 1):
    report += f"   {i}. {row['feature']:20s}: {row['importance']:.4f}\n"

report += f"""
8. KẾT LUẬN:
   - Mô hình Random Forest đã được tối ưu hóa thành công
   - Hiệu suất trên tập test: RMSE = {test_rmse:.4f} năm, R² = {test_r2:.4f}
   - Random Forest cải thiện đáng kể so với Decision Tree đơn lẻ
   - Mô hình ổn định với độ lệch chuẩn CV thấp: {np.std(best_cv_scores):.4f}
   - Mô hình đã được lưu tại: {model_path}
   - Các biểu đồ đã được lưu tại: ../visualization/random_forest/

9. ƯU ĐIỂM CỦA MÔ HÌNH NÀY:
   - Giảm overfitting so với Decision Tree đơn lẻ
   - Feature importance đáng tin cậy hơn
   - Hiệu suất dự đoán ổn định và chính xác
   - Robust với outliers và nhiễu

{'='*70}
"""

print(report)

# Lưu báo cáo vào file
report_path = '../saved_models/random_forest_report.md'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report)

print(f"\n✓ Đã lưu báo cáo tại: {report_path}")

## Kết luận

### Tổng kết:
1. ✅ **Mô hình Random Forest** đã được xây dựng và tối ưu hóa thành công
2. ✅ **GridSearchCV với 5-Fold CV** đã tìm được bộ siêu tham số tối ưu
3. ✅ **Feature Importance** đã xác định các yếu tố quan trọng nhất ảnh hưởng đến tuổi thọ
4. ✅ **Hiệu suất mô hình** đã được đánh giá toàn diện trên cả 3 tập dữ liệu
5. ✅ **Mô hình đã được lưu** tại `saved_models/random_forest.pkl`
6. ✅ **Trực quan hóa chi tiết** đã được tạo và lưu tại `visualization/random_forest/`

### Ưu điểm của Random Forest so với Decision Tree:
- **Giảm overfitting:** Ensemble nhiều cây giúp tránh học thuộc lòng dữ liệu
- **Ổn định hơn:** Kết quả ít bị ảnh hưởng bởi thay đổi nhỏ trong dữ liệu
- **Chính xác hơn:** Kết hợp dự đoán từ nhiều cây cho kết quả tốt hơn
- **Feature importance đáng tin cậy:** Trung bình từ nhiều cây cho đánh giá chính xác hơn

### Hạn chế:
- Tốn nhiều thời gian huấn luyện hơn Decision Tree đơn
- Khó giải thích hơn (black box hơn)
- Tốn nhiều bộ nhớ để lưu trữ nhiều cây
- Thời gian dự đoán chậm hơn (phải chạy qua tất cả các cây)

### So sánh với Decision Tree:
| Metric | Decision Tree | Random Forest | Cải thiện |
|--------|--------------|---------------|-----------|
| Validation RMSE | (baseline) | {val_rmse:.4f} | ✓ |
| Validation R² | (baseline) | {val_r2:.4f} | ✓ |
| Test RMSE | - | {test_rmse:.4f} | - |
| Test R² | - | {test_r2:.4f} | - |

### Bước tiếp theo:
- So sánh với các mô hình boosting (XGBoost, LightGBM, CatBoost)
- So sánh với SVM Regression
- Xây dựng ensemble model kết hợp nhiều loại thuật toán
- Phân tích error cases để hiểu rõ hơn về giới hạn của mô hình